In [176]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import numpy as np
import spacy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Bidirectional

In [177]:
df = pd.read_json('./cleaned_kaggle_data.json', encoding='utf-8', lines=True)

In [178]:
df['category'] = df['category'].replace("U.S. NEWS", "US NEWS")

In [179]:
df.head()

,link,headline,category,short_description,authors,date,cleaned_text,text
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,US NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23,4 million Americans roll sleeve Omicron target...,Over 4 Million Americans Roll Up Sleeves For O...
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",US NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23,American Airlines Flyer charge ban life Punchi...,"American Airlines Flyer Charged, Banned For Li..."
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23,23 Funniest tweet Cats Dogs Week Sept. 17 23 d...,23 Of The Funniest Tweets About Cats And Dogs ...
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23,Funniest Tweets parent Week Sept. 17 23 accide...,The Funniest Tweets From Parents This Week (Se...
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,US NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22,woman call cop Black Bird Watcher lose Lawsuit...,Woman Who Called Cops On Black Bird-Watcher Lo...


In [180]:
le = LabelEncoder()
df['category_encoded'] = le.fit_transform(df['category'])

In [181]:
# Remember the encoding values
le.classes_

array(['ARTS & CULTURE', 'BLACK VOICES', 'BUSINESS', 'COLLEGE', 'COMEDY',
       'CRIME', 'DIVORCE', 'EDUCATION', 'ENTERTAINMENT', 'ENVIRONMENT',
       'FIFTY', 'FOOD & DRINK', 'GOOD NEWS', 'GREEN', 'HEALTHY LIVING',
       'HOME & LIVING', 'IMPACT', 'LATINO VOICES', 'MEDIA', 'MONEY',
       'PARENTING', 'PARENTS', 'POLITICS', 'QUEER VOICES', 'RELIGION',
       'SCIENCE', 'SPORTS', 'STYLE & BEAUTY', 'TASTE', 'TECH', 'TRAVEL',
       'US NEWS', 'WEDDINGS', 'WEIRD NEWS', 'WELLNESS', 'WOMEN',
       'WORLD NEWS', 'WORLDPOST'], dtype=object)

In [182]:
nlp = spacy.load('en_core_web_lg')

In [183]:
# this is how I counted the vocabulary of my training data
# unique_words = set()
# df['cleaned_text'].apply(lambda x: unique_words.update([token.text.lower() for token in nlp(x) if token.is_alpha]))
# vocab_size = len(unique_words)
max_vocab_size = 79360
trunc_type='post'
padding_type='post'

In [184]:
# Calculate the length of each string in the column
df['lengths'] = df['cleaned_text'].apply(len)

# Find the maximum length
max_length = df['lengths'].max()

In [200]:
X = df['cleaned_text'].values
labels = df['category_encoded'].values

# Convert labels to categorical format
y = tf.keras.utils.to_categorical(labels, num_classes=38)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
vocabulary_size = len(tokenizer.word_counts)
X_train, X_test, y_train, y_test = train_test_split(np.array(padded_sequences), np.array(y), test_size=0.33, random_state=42)

In [202]:
model = Sequential([
    Embedding(input_dim=vocabulary_size, output_dim=64),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dense(38, activation='softmax')
])

In [203]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_15 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_15                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [204]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
 164/4387 ━━━━━━━━━━━━━━━━━━━━ 21:08 300ms/step - accuracy: 0.1546 - loss: 3.3512

KeyboardInterrupt: 

In [ ]:
def pre_process_document(text):
    doc = nlp(text)
    filtered_tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(filtered_tokens)

In [ ]:
def preprocess_text(texts, tokenizer, max_length):
    """
    Function to preprocess new text inputs.
    
    Args:
    texts (list of str): New texts to predict.
    tokenizer (Tokenizer): Fitted Keras tokenizer.
    max_length (int): Maximum length for padding.
    
    Returns:
    array: Preprocessed texts suitable for model input.
    """
    # Convert texts to sequences
    sequences = tokenizer.texts_to_sequences(texts)
    
    # Pad the sequences
    padded = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
    
    return padded


In [206]:
sample_text = pre_process_document("The movie was cool and the graphics were out of this world")
new_sequences = preprocess_text(sample_text, tokenizer, max_length)
predictions = model.predict(new_sequences)

ValueError: Invalid dtype: str768